In [14]:
import pandas as pd

# Memuat dataset
file_path = '/content/heart.csv'
data = pd.read_csv(file_path)

# Menampilkan beberapa baris pertama untuk memahami struktur data
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


Dataset berhasil dimuat dan memiliki beberapa kolom dengan informasi terkait karakteristik pasien (seperti usia, jenis kelamin, tekanan darah, dll.) serta kolom target yang menunjukkan apakah pasien memiliki penyakit jantung (0: tidak, 1: ya)

**Langkah berikutnya adalah mengecek apakah ada nilai yang hilang di dataset ini**

In [20]:
# Mengecek kembali apakah ada nilai yang hilang dalam dataset
missing_values = dataset.isnull().sum()

# Menampilkan jumlah nilai yang hilang untuk setiap kolom
missing_values

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


Dataset tidak memiliki nilai yang hilang pada kolom manapun, sehingga kita dapat langsung melanjutkan ke langkah preprocessing data. Berikutnya, kita akan membagi dataset menjadi fitur (input) dan target (output), serta melakukan normalisasi fitur untuk memastikan pelatihan model lebih stabil.

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Memisahkan fitur (X) dan target (y)
X = dataset.drop('target', axis=1)
y = dataset['target']

# Membagi data menjadi set pelatihan dan pengujian (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalisasi fitur dengan StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Menampilkan dimensi dataset setelah pembagian
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((820, 13), (205, 13), (820,), (205,))

Dataset telah berhasil dibagi menjadi data pelatihan (80%) dan pengujian (20%). Data fitur telah dinormalisasi menggunakan StandardScaler untuk memastikan setiap fitur memiliki mean 0 dan standar deviasi 1. Berikut dimensi dataset:

X_train: 820 sampel, 13 fitur.

X_test: 205 sampel, 13 fitur.

y_train dan y_test: masing-masing target yang sesuai.

**Langkah selanjutnya adalah mendefinisikan model MLP menggunakan PyTorch.**

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Konversi data menjadi tensor PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Membuat DataLoader untuk batch processing
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Mendefinisikan arsitektur model MLP
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Layer input ke hidden
        self.relu = nn.ReLU()  # Aktivasi ReLU
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Layer hidden ke hidden
        self.fc3 = nn.Linear(hidden_size, output_size)  # Layer hidden ke output
        self.softmax = nn.Softmax(dim=1)  # Aktivasi softmax untuk output

    def forward(self, x):
        x = self.fc1(x)  # Masukkan ke layer pertama
        x = self.relu(x)  # Aktivasi ReLU
        x = self.fc2(x)  # Masukkan ke layer kedua
        x = self.relu(x)  # Aktivasi ReLU
        x = self.fc3(x)  # Masukkan ke layer output
        return x  # Output logits

# Parameter model
input_size = X_train.shape[1]  # Jumlah fitur
hidden_size = 64  # Jumlah unit pada hidden layer
output_size = 2  # Jumlah kelas (0 dan 1)

# Inisialisasi model
model = MLPModel(input_size, hidden_size, output_size)

# Fungsi loss dan optimizer
criterion = nn.CrossEntropyLoss()  # Loss untuk klasifikasi
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizer Adam

# Menampilkan arsitektur model
model


MLPModel(
  (fc1): Linear(in_features=13, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

Model MLP ini terdiri dari 3 layer utama:

Input layer (fc1): Menerima 13 fitur dan menghasilkan 64 neuron.

Hidden layer (fc2): Memproses 64 neuron menjadi 64 neuron dengan aktivasi ReLU.

Output layer (fc3): Menghasilkan 2 kelas (output) menggunakan Softmax untuk probabilitas.

Struktur ini cukup sederhana untuk dataset ukuran kecil-menengah. ReLU menambah non-linearitas, sementara output Softmax memastikan hasil berupa probabilitas.

**Bandingkan Hidden Layer (1,2,3) dengan jumlah neuron (4, 8, 16, 32, 64,...)**

In [24]:
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import pandas as pd

# Fungsi untuk membuat model MLP dengan parameter fleksibel
class FlexibleMLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(FlexibleMLP, self).__init__()
        self.hidden_layers = nn.ModuleList()  # List untuk menyimpan hidden layers
        prev_size = input_size  # Ukuran input awal
        for hidden_size in hidden_sizes:
            self.hidden_layers.append(nn.Linear(prev_size, hidden_size))
            prev_size = hidden_size
        self.output_layer = nn.Linear(prev_size, output_size)  # Output layer

    def forward(self, x):
        for layer in self.hidden_layers:
            x = F.relu(layer(x))  # Aktivasi ReLU untuk setiap hidden layer
        x = self.output_layer(x)  # Layer output tanpa aktivasi softmax (digunakan dalam CrossEntropyLoss)
        return x

# Fungsi untuk melatih model dan mengukur akurasi
def train_and_evaluate(hidden_layers, neurons_per_layer):
    # Inisialisasi model dengan konfigurasi tertentu
    model = FlexibleMLP(input_size=X_train.shape[1], hidden_sizes=[neurons_per_layer] * hidden_layers, output_size=2)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Pelatihan model
    epochs = 10  # Kurangi epoch untuk mempercepat eksperimen
    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()  # Reset gradient
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Hitung loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update bobot model

    # Evaluasi model pada data pengujian
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    # Mengukur akurasi
    acc = accuracy_score(y_test, predictions)
    return acc

# Parameter eksperimen
hidden_layer_options = [1, 2, 3]  # Jumlah hidden layers
neurons_per_layer_options = [4, 8, 16, 32, 64]  # Jumlah neuron per layer

# Menyimpan hasil eksperimen
results = []

# Looping untuk setiap kombinasi jumlah hidden layers dan jumlah neuron
for hidden_layers in hidden_layer_options:
    for neurons_per_layer in neurons_per_layer_options:
        acc = train_and_evaluate(hidden_layers, neurons_per_layer)
        results.append({"Hidden Layers": hidden_layers, "Neurons per Layer": neurons_per_layer, "Accuracy": acc})

# Menyimpan hasil dalam DataFrame untuk analisis
results_df = pd.DataFrame(results)

# Menampilkan hasil eksperimen sebagai tabel
print(results_df)

# Menyimpan hasil ke file CSV untuk analisis lebih lanjut
results_df.to_csv("hasil_perbandingan_hidden_layers.csv", index=False)

# Jika Anda di Google Colab, unduh file CSV
from google.colab import files
files.download("hasil_perbandingan_hidden_layers.csv")


    Hidden Layers  Neurons per Layer  Accuracy
0               1                  4  0.834146
1               1                  8  0.780488
2               1                 16  0.824390
3               1                 32  0.858537
4               1                 64  0.843902
5               2                  4  0.848780
6               2                  8  0.819512
7               2                 16  0.848780
8               2                 32  0.848780
9               2                 64  0.878049
10              3                  4  0.839024
11              3                  8  0.843902
12              3                 16  0.873171
13              3                 32  0.897561
14              3                 64  0.951220


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**1.Pengaruh Jumlah Hidden Layers**

**1 Hidden Layer:**

*   Akurasi bervariasi dari 0.780488 hingga 0.858537 saat jumlah neuron meningkat.
*   Meskipun akurasi meningkat saat jumlah neuron bertambah, performanya terbatas dibandingkan konfigurasi dengan lebih banyak hidden layer.

**2 Hidden Layers:**

*   Akurasi meningkat dibandingkan dengan konfigurasi 1 hidden layer, mulai dari 0.819512 hingga 0.878049.
*   Model lebih mampu menangkap pola data karena memiliki representasi yang lebih kompleks.

**3 Hidden Layers:**

*   Memberikan hasil terbaik dengan akurasi tertinggi mencapai 0.951220 saat jumlah neuron adalah 64.
*   Dengan lebih banyak hidden layer, model memiliki kapasitas lebih besar untuk mempelajari pola data yang kompleks.

**2.Pengaruh Jumlah Neuron per Layer**

*   Pada setiap jumlah hidden layer, penambahan jumlah neuron cenderung meningkatkan akurasi hingga titik tertentu.
*   64 neuron per layer memberikan hasil terbaik di semua konfigurasi, terutama pada model dengan 3 hidden layers.

**3.Konfigurasi Optimal**

*   3 Hidden Layers dengan 64 Neuron per Layer memberikan akurasi tertinggi (0.951220).
*    Ini menunjukkan bahwa model dengan lebih banyak layer dan neuron lebih baik dalam menangkap pola yang kompleks pada dataset ini.

**4.Trade-Offs**


*   Model dengan 3 hidden layers dan lebih banyak neuron memerlukan lebih banyak waktu pelatihan dan sumber daya komputasi.
*   Jika sumber daya terbatas, 2 Hidden Layers dengan 32 atau 64 Neuron per Layer adalah alternatif yang baik karena memberikan akurasi tinggi (0.878049) dengan kompleksitas lebih rendah.

**Rekomendasi:**


*   Gunakan 3 Hidden Layers dengan 64 Neuron per Layer untuk performa terbaik jika sumber daya mendukung.
*   Untuk efisiensi, pilih 2 Hidden Layers dengan 32 Neuron per Layer, yang masih memberikan akurasi tinggi tanpa terlalu membebani komputasi.





**Bandingkan Activation Function (linear,Sigmoid,RelU,Softmax,Tanh)**

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Data Preprocessing
dataset = pd.read_csv("heart.csv")
X = dataset.drop("target", axis=1)
y = dataset["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)
batch_size = 32
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size, shuffle=False)

# Definisi Model Flexible dengan Fungsi Aktivasi
class ActivationComparisonMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation_fn):
        super(ActivationComparisonMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.activation_fn = activation_fn

    def forward(self, x):
        x = self.activation_fn(self.fc1(x))
        x = self.activation_fn(self.fc2(x))
        x = self.fc3(x)
        return x

# Fungsi untuk melatih model dan mengukur akurasi
def train_and_evaluate_activation(activation_fn):
    model = ActivationComparisonMLP(input_size=X_train.shape[1], hidden_size=32, output_size=2, activation_fn=activation_fn)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Pelatihan model
    for epoch in range(10):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluasi model
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    # Mengukur akurasi
    return accuracy_score(y_test, predictions)

# Membandingkan berbagai fungsi aktivasi
activation_functions = {
    "Linear": lambda x: x,
    "Sigmoid": torch.sigmoid,
    "ReLU": F.relu,
    "Softmax": lambda x: F.softmax(x, dim=1),
    "Tanh": torch.tanh
}

results = []
for name, activation_fn in activation_functions.items():
    accuracy = train_and_evaluate_activation(activation_fn)
    results.append({"Activation Function": name, "Accuracy": accuracy})

# Menampilkan hasil
results_df = pd.DataFrame(results)
print(results_df)


  Activation Function  Accuracy
0              Linear  0.814634
1             Sigmoid  0.804878
2                ReLU  0.873171
3             Softmax  0.756098
4                Tanh  0.834146


**Kesimpulan:**

ReLU adalah fungsi aktivasi terbaik pada eksperimen ini, menghasilkan akurasi tertinggi (0.873171).

Tanh adalah alternatif yang baik untuk data terpusat, tetapi sedikit kalah dengan ReLU.

Sigmoid dan Softmax memiliki performa yang lebih rendah karena masalah saturasi dan tidak optimal untuk hidden layers.

Linear hanya memberikan performa dasar tanpa manfaat non-linearitas.

**Rekomendasi:**

Gunakan ReLU untuk hidden layers pada sebagian besar kasus karena kinerjanya yang cepat dan stabil. Hindari penggunaan Softmax di hidden layers, karena lebih cocok untuk output layer pada klasifikasi multi-kelas. Jika data sangat terpusat, Tanh bisa menjadi alternatif.

**Bandingkan Epoch (1,10,25,50,100,250)**

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Data Preprocessing
dataset = pd.read_csv("heart.csv")  # Pastikan dataset "heart.csv" tersedia
X = dataset.drop("target", axis=1)
y = dataset["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

# Definisi Model
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Layer pertama
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Layer kedua
        self.fc3 = nn.Linear(hidden_size, output_size)  # Layer output
        self.relu = nn.ReLU()  # Aktivasi ReLU

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Aktivasi setelah layer pertama
        x = self.relu(self.fc2(x))  # Aktivasi setelah layer kedua
        x = self.fc3(x)  # Output tanpa aktivasi softmax (CrossEntropyLoss akan menangani ini)
        return x

# Fungsi Pelatihan dan Evaluasi
def train_and_evaluate_epochs(num_epochs):
    # Inisialisasi model, optimizer, dan loss function
    model = SimpleMLP(input_size=X_train.shape[1], hidden_size=32, output_size=2)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Pelatihan model
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()  # Reset gradient
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Hitung loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update bobot model

    # Evaluasi model
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # Prediksi kelas dengan probabilitas tertinggi
            predictions.extend(predicted.cpu().numpy())

    # Menghitung akurasi
    return accuracy_score(y_test, predictions)

# Membandingkan berbagai epoch
epoch_options = [1, 10, 25, 50, 100, 250]
results = []

for epochs in epoch_options:
    accuracy = train_and_evaluate_epochs(epochs)
    results.append({"Epochs": epochs, "Accuracy": accuracy})

# Menampilkan hasil
results_df = pd.DataFrame(results)
print(results_df)


   Epochs  Accuracy
0       1  0.800000
1      10  0.843902
2      25  0.941463
3      50  1.000000
4     100  1.000000
5     250  1.000000


**Kesimpulan:**

Akurasi meningkat seiring bertambahnya jumlah epoch hingga mencapai konvergensi pada 50 epoch.

Setelah 50 epoch, model sudah cukup terlatih, dan penambahan epoch tidak meningkatkan akurasi.

Epoch yang terlalu sedikit (1 atau 10) tidak cukup untuk melatih model secara optimal.

50 epoch adalah titik optimal untuk pelatihan pada dataset ini, karena mencapai akurasi maksimal dengan efisiensi pelatihan.

**Rekomendasi:**

Gunakan 50 epoch sebagai jumlah epoch yang optimal untuk pelatihan model ini, karena sudah cukup untuk mencapai akurasi sempurna.
Hindari penggunaan epoch terlalu tinggi (>100) kecuali diperlukan untuk dataset yang lebih kompleks, karena dapat membuang waktu komputasi.

**Bandingkan Learning Rate (10/1/0.1/0.01/0.001/0.0001)**

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Data Preprocessing
dataset = pd.read_csv("heart.csv")  # Pastikan dataset "heart.csv" tersedia
X = dataset.drop("target", axis=1)
y = dataset["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

# Definisi Model
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Layer pertama
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Layer kedua
        self.fc3 = nn.Linear(hidden_size, output_size)  # Layer output
        self.relu = nn.ReLU()  # Aktivasi ReLU

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Aktivasi setelah layer pertama
        x = self.relu(self.fc2(x))  # Aktivasi setelah layer kedua
        x = self.fc3(x)  # Output tanpa aktivasi softmax (CrossEntropyLoss akan menangani ini)
        return x

# Fungsi Pelatihan dan Evaluasi
def train_and_evaluate_lr(learning_rate):
    # Inisialisasi model, optimizer, dan loss function
    model = SimpleMLP(input_size=X_train.shape[1], hidden_size=32, output_size=2)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Optimizer dengan learning rate spesifik
    criterion = nn.CrossEntropyLoss()

    # Pelatihan model
    for epoch in range(50):  # Tetapkan jumlah epoch untuk semua eksperimen
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()  # Reset gradient
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Hitung loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update bobot model

    # Evaluasi model
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # Prediksi kelas dengan probabilitas tertinggi
            predictions.extend(predicted.cpu().numpy())

    # Menghitung akurasi
    return accuracy_score(y_test, predictions)

# Membandingkan berbagai learning rate
learning_rate_options = [10, 1, 0.1, 0.01, 0.001, 0.0001]  # Daftar nilai learning rate
results = []

for lr in learning_rate_options:
    accuracy = train_and_evaluate_lr(lr)  # Melatih model dengan learning rate tertentu
    results.append({"Learning Rate": lr, "Accuracy": accuracy})  # Simpan hasil

# Menampilkan hasil
results_df = pd.DataFrame(results)
print(results_df)


   Learning Rate  Accuracy
0        10.0000  0.580488
1         1.0000  0.531707
2         0.1000  0.751220
3         0.0100  1.000000
4         0.0010  0.990244
5         0.0001  0.848780


**Kesimpulan:**

Learning Rate = 0.01 adalah yang terbaik untuk eksperimen ini, menghasilkan akurasi sempurna 1.000000.

Learning Rate yang terlalu besar (10.0, 1.0) menyebabkan model gagal belajar (divergensi).

Learning Rate yang terlalu kecil (0.0001) membuat proses pembelajaran terlalu lambat dan tidak optimal.

0.001 adalah alternatif yang baik jika stabilitas lebih diutamakan.

**Rekomendasi:**

Gunakan Learning Rate = 0.01 untuk performa optimal.

Jika dataset lebih kompleks atau model lebih besar, pertimbangkan 0.001 untuk stabilitas, meskipun memerlukan lebih banyak epoch untuk mencapai konvergensi.

**Bandingkan Batch Size (16,32,64,128,256,512)**

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Data Preprocessing
dataset = pd.read_csv("heart.csv")  # Pastikan dataset "heart.csv" tersedia
X = dataset.drop("target", axis=1)
y = dataset["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Definisi Model
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Layer pertama
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Layer kedua
        self.fc3 = nn.Linear(hidden_size, output_size)  # Layer output
        self.relu = nn.ReLU()  # Aktivasi ReLU

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Aktivasi setelah layer pertama
        x = self.relu(self.fc2(x))  # Aktivasi setelah layer kedua
        x = self.fc3(x)  # Output tanpa aktivasi softmax (CrossEntropyLoss akan menangani ini)
        return x

# Fungsi Pelatihan dan Evaluasi
def train_and_evaluate_batch_size(batch_size):
    # Membuat DataLoader dengan batch size yang berbeda
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size, shuffle=False)

    # Inisialisasi model, optimizer, dan loss function
    model = SimpleMLP(input_size=X_train.shape[1], hidden_size=32, output_size=2)
    optimizer = optim.Adam(model.parameters(), lr=0.01)  # Learning rate tetap
    criterion = nn.CrossEntropyLoss()

    # Pelatihan model
    for epoch in range(50):  # Tetapkan jumlah epoch tetap
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()  # Reset gradient
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Hitung loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update bobot model

    # Evaluasi model
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # Prediksi kelas dengan probabilitas tertinggi
            predictions.extend(predicted.cpu().numpy())

    # Menghitung akurasi
    return accuracy_score(y_test, predictions)

# Membandingkan berbagai batch size
batch_size_options = [16, 32, 64, 128, 256, 512]  # Daftar batch size
results = []

for batch_size in batch_size_options:
    accuracy = train_and_evaluate_batch_size(batch_size)  # Melatih model dengan batch size tertentu
    results.append({"Batch Size": batch_size, "Accuracy": accuracy})  # Simpan hasil

# Menampilkan hasil
results_df = pd.DataFrame(results)
print(results_df)


   Batch Size  Accuracy
0          16       1.0
1          32       1.0
2          64       1.0
3         128       1.0
4         256       1.0
5         512       1.0


**Kesimpulan:**

Batch size tidak memengaruhi akurasi dalam eksperimen ini karena model mencapai akurasi sempurna di semua konfigurasi.

Batch size yang optimal tergantung pada sumber daya komputasi:
Gunakan batch size besar (128 atau 256) untuk pelatihan lebih cepat jika memori GPU cukup.

Gunakan batch size kecil (16 atau 32) jika memori terbatas atau untuk dataset yang sangat bervariasi.

**Rekomendasi:**

Pilih batch size sesuai kapasitas hardware (GPU/CPU). Untuk kasus ini, Batch Size = 128 adalah kompromi terbaik antara efisiensi dan kecepatan pelatihan.

Jika dataset lebih kompleks atau lebih besar, batch size yang lebih besar (256 atau 512) bisa lebih efisien dengan memanfaatkan GPU.







**Bandingkan Hidden Layer, Activation Function, Epoch, Learning Rate, Batch Size**

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

# Data Preprocessing
dataset = pd.read_csv("heart.csv")  # Pastikan dataset "heart.csv" tersedia
X = dataset.drop("target", axis=1)
y = dataset["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Definisi Model
class FlexibleMLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation_fn):
        super(FlexibleMLP, self).__init__()
        self.hidden_layers = nn.ModuleList()
        prev_size = input_size
        for hidden_size in hidden_sizes:
            self.hidden_layers.append(nn.Linear(prev_size, hidden_size))
            prev_size = hidden_size
        self.output_layer = nn.Linear(prev_size, output_size)
        self.activation_fn = activation_fn

    def forward(self, x):
        for layer in self.hidden_layers:
            x = self.activation_fn(layer(x))
        x = self.output_layer(x)
        return x

# Fungsi Pelatihan dan Evaluasi
def train_and_evaluate(hidden_layers, activation_fn, epochs, learning_rate, batch_size):
    # Membuat DataLoader dengan batch size yang diberikan
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size, shuffle=False)

    # Inisialisasi model
    model = FlexibleMLP(input_size=X_train.shape[1], hidden_sizes=[32] * hidden_layers, output_size=2, activation_fn=activation_fn)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Pelatihan model
    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluasi model
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    # Menghitung akurasi
    return accuracy_score(y_test, predictions)

# Parameter yang akan dibandingkan
hidden_layer_options = [1, 2, 3]
activation_functions = {
    "ReLU": F.relu,
    "Sigmoid": torch.sigmoid,
    "Tanh": torch.tanh
}
epoch_options = [10, 50]
learning_rate_options = [0.01, 0.001]
batch_size_options = [32, 128]

# Menyimpan hasil eksperimen
results = []

# Looping untuk semua kombinasi parameter
for hidden_layers in hidden_layer_options:
    for activation_name, activation_fn in activation_functions.items():
        for epochs in epoch_options:
            for learning_rate in learning_rate_options:
                for batch_size in batch_size_options:
                    accuracy = train_and_evaluate(hidden_layers, activation_fn, epochs, learning_rate, batch_size)
                    results.append({
                        "Hidden Layers": hidden_layers,
                        "Activation Function": activation_name,
                        "Epochs": epochs,
                        "Learning Rate": learning_rate,
                        "Batch Size": batch_size,
                        "Accuracy": accuracy
                    })

# Menampilkan hasil
results_df = pd.DataFrame(results)
print(results_df)

    Hidden Layers Activation Function  Epochs  Learning Rate  Batch Size  \
0               1                ReLU      10          0.010          32   
1               1                ReLU      10          0.010         128   
2               1                ReLU      10          0.001          32   
3               1                ReLU      10          0.001         128   
4               1                ReLU      50          0.010          32   
..            ...                 ...     ...            ...         ...   
67              3                Tanh      10          0.001         128   
68              3                Tanh      50          0.010          32   
69              3                Tanh      50          0.010         128   
70              3                Tanh      50          0.001          32   
71              3                Tanh      50          0.001         128   

    Accuracy  
0   0.960976  
1   0.892683  
2   0.829268  
3   0.824390  
4   1.000000

**Kesimpulan dari Hasil Eksperimen:**
Hidden Layers:

1 Hidden Layer: Memberikan performa yang baik pada kombinasi parameter optimal, tetapi kurang fleksibel untuk pola yang lebih kompleks.

2 Hidden Layers: Lebih stabil dan secara konsisten memberikan akurasi tinggi di berbagai kombinasi parameter.

3 Hidden Layers: Memberikan akurasi tertinggi pada data yang lebih kompleks, tetapi memerlukan lebih banyak sumber daya komputasi dan tuning parameter.
Activation Function:

ReLU adalah fungsi aktivasi terbaik secara keseluruhan, memberikan akurasi tertinggi secara konsisten dengan stabilitas yang baik.

Tanh: Alternatif yang baik pada beberapa kombinasi, tetapi kurang stabil dibandingkan ReLU.

Sigmoid: Tidak optimal karena masalah saturasi dan gradien yang menghilang, sering memberikan akurasi lebih rendah dibandingkan ReLU.

Epochs:

10 Epochs: Tidak cukup untuk melatih model secara optimal, menghasilkan akurasi yang lebih rendah.

50 Epochs: Memberikan akurasi maksimal pada sebagian besar kombinasi parameter, cukup untuk mencapai konvergensi.

Learning Rate:

0.01: Memberikan hasil terbaik pada sebagian besar kombinasi parameter.

0.001: Stabil, tetapi memerlukan lebih banyak epoch untuk mencapai performa maksimal.

Batch Size:

Batch size 32 dan 128 memberikan hasil yang konsisten baik tanpa memengaruhi akurasi secara signifikan.

Batch size lebih kecil memungkinkan pembaruan model lebih sering tetapi membutuhkan lebih banyak iterasi.

**Rekomendasi Akhir:**
Gunakan 2 atau 3 hidden layers untuk fleksibilitas dan performa yang lebih baik pada pola data yang kompleks.

Gunakan ReLU sebagai fungsi aktivasi utama karena memberikan stabilitas dan performa tertinggi.

Pilih 50 epochs untuk memastikan model terlatih secara optimal.

Gunakan learning rate = 0.01 untuk mencapai performa maksimal dengan stabilitas.

Pilih batch size 32 atau 128 sesuai dengan sumber daya komputasi yang tersedia.